In [19]:
import torch
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from sklearn.model_selection import train_test_split
from visdom import Visdom
import torch.nn.functional as F
from torch.nn.init import xavier_normal_
%run Utility_Functions.ipynb
%run parameters.py

In [20]:
# Data import 
x = np.load('X_Multi_Class.npy')
y = np.load('Y_Multi_Class.npy')

# Balanced the dataset 
x_balanced, y_balanced = _make_balanced(x, y)

x = x_balanced
y = y_balanced

print(x.shape, y.shape, sum(y)/len(y))
print(np.min(x), np.max(x), np.mean(x))

# Visualization
viz = Visdom()
opts = dict(legend=['Training','Validation','Testing'], xlabel = 'Iteration', ylabel = 'Accuracy', title = 'EEG Classification')
# Device configuration 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
device # verify which device is being used

(8224, 20, 256) (8224, 3) [0.35907101 0.33110409 0.3098249 ]
-127.35785764448343 120.9554109493638 1.1879818356883972e-19


device(type='cuda', index=0)

In [28]:
# Prepare the data for convolution neural network 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.40)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size = 0.50)


class custom_dataset(Dataset):
    def __init__(self, x, y):
        super(custom_dataset, self).__init__()
        self.x = torch.from_numpy(x).type(torch.float32)
        self.y = torch.from_numpy(y).type(torch.float32)
    
    def __getitem__(self, ids):
        return self.x[ids], self.y[ids] # 2 index of total force ir index
    
    def __len__(self):
        return self.x.shape[0]

# Create datasets 
train_data = custom_dataset(x_train, y_train)
valid_data = custom_dataset(x_valid, y_valid)
test_data = custom_dataset(x_test, y_test)
    
# Hyper-parameters 
OUTPUT = 3
NUM_EPOCHS = 500
BATCH_SIZE = 64
LEARNING_RATE = 0.001
# WEIGHT_DECAY = 0.0001
BIAS = True
    
# Load datasets 
train_data_iterator = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_data_iterator = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=True)
test_data_iterator = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier_normal_(m.weight.data)
        
def accuracy(data_iterator):
    with torch.no_grad():
        total = 0
        length = 0
        for x, y in data_iterator:
            out_put = model(x.to(device))
            out_put = out_put.cpu().detach()
            total += (out_put.argmax(dim=1)==y.argmax(dim=1)).float().sum()
            length += len(y)
        accuracy = total/length
        return accuracy

class ConvNet(nn.Module):
    def __init__(self, OUTPUT):
        super(ConvNet, self).__init__()
        self.net_1 = nn.Sequential(
            nn.Conv2d(1, 20, kernel_size=(1, 15), stride=1, bias=False),
            nn.Conv2d(20, 20, kernel_size=(10, 10), stride=1, bias=False),
            nn.BatchNorm2d(20, momentum=0.1, affine=True)
            )
        
        self.pool = nn.AvgPool2d(kernel_size=(1, 75), stride=(1, 15))
        
        self.net_2 = nn.Sequential(
            nn.Conv2d(20, OUTPUT, kernel_size=(11, 11), stride=1),
            nn.LogSoftmax()
            )
        
    def forward(self, x):
        x = x.view(-1, 1, n_electrodes, epoch_length*s_freq)
        out = self.net_1(x)  
        out = out*out
        out = self.pool(out)
        out = torch.log(torch.clamp(out, min=1e-6))
        out = self.net_2(out)
        out = torch.squeeze(out)
    
        return out
    
# An instance of model
model = ConvNet(OUTPUT).to(device)
model.apply(weights_init)

viz.close(win='Accuracy')
  
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

i = 0

for epoch in range(NUM_EPOCHS):
    for x_batch, y_batch in train_data_iterator:
        # Send the images and labels to gpu 
        x_batch = x_batch.to(device)
        y_batch = (torch.max(y_batch, dim=1)[1]).to(device)

        # Forward pass
        out_put = model(x_batch) 
        loss = criterion(out_put, y_batch)

        # Backward and optimize
        optimizer.zero_grad() # For batch gradient optimisation
        loss.backward()
        optimizer.step()
        
# Calculate the accuracy
    train_accuracy = accuracy(train_data_iterator)
    valid_accuracy = accuracy(valid_data_iterator)
    test_accuracy = accuracy(test_data_iterator)

    i = i + 1 
    viz.line(X = np.column_stack([i]*3), Y = np.column_stack((train_accuracy, valid_accuracy, test_accuracy)), opts = opts, win='Accuracy', update='append')